# NLP Text Analytics Splunk App - Blog
##### Author: Nathan Worsham
##### Created for MSDS692 Data Science Practicum I at Regis University, 2018

Most other projects for the Data Science Practicum revolved around collecting a dataset, cleaning it, exploring it, and then using machine learning algorithms on it to perform some sort of prediction, classification, or extrapolate some analysis. I decided to go a bit outside the box and instead build a Splunk App for my project (though the app still ends up following these familiar steps in its own ways).

## Inspiration
Let me first preface by stating I am not a developer... but I don't let that stop me. Like many of my cohorts, I have always worked in information technology but I have a completely unrelated background--music. I tinker, I experiment, I'm curious, I push and push until I find an answer. I don't always know the correct or proper methodologies but I am willing to learn. I have however built a few Splunk Apps previously (using the same techniques just mentioned) so I was at least familiar with the practice and just before this class, I took a class on text analytics. I am fascinated with Splunk, Python, and the subject of text analytics. I already had a few (internal) dashboards taking a shot at text analytics if only mildly. After the course though, I realized some of the shortcomings that Splunk had regarding text analytics. Granted system logs are not something you think of immediately having natural language (some actually do) but that is not the only source of text in Splunk as it can consume anything human readable. In my own place of work, one immediate example that stood out in my mind was support ticket summaries and descriptions but I'm sure the sources do not stop there. 

## Description
So given that information, the intent of this project is create a Splunk app that will provide a simple interface for analyzing any text a user points at it in Splunk using Python natural language processing libraries (currently just NLTK 3.3). The app provides custom commands and dashboards to show how to use it and walk a user through text analytics.

## The Details
This project represents existing knowledge I already have in Splunk (regarding developing apps, administration, using SPL, building dashboards and I'm sure more) and building upon it. Splunk apps that want to use new Python libraries generally get packaged with those additional libraries needed to run the custom commands within the app, doing so by importing locally--when you say `import` in Python, the first place it looks is in the current working directory. This of course makes sense as Splunk doesn't want to break their existing Python environment by allowing admins or developers to add to and change it permanently. Splunk also supplies a Python SDK (http://dev.splunk.com/python) which takes care of many remedial tasks and keeps things consistent. I will be relying on this heavily and using the documenting standards they give in their examples within the SDK which are a bit different then say sphinx markup or at least I think so. 

#### Getting NLTK Working

As far as Python libraries go for natural language processing there are really two major contenders spaCy (https://spacy.io) and nltk or _natural language toolkit_ (http://www.nltk.org). I really like the features and performance of spaCy but during this project I found one major downfall for spaCy. Since it is is a C extension--simple way to tell is if the package extension is `.so`--it would have to be compiled for the host system first before it can be imported locally. Is this an impossible step to overcome? No, I'm sure it is not. However this practicum class is only 8 weeks long, so time is of the essence. Relatedly, early on in the process I decided to go with the concept of MVP or minimum viable product, to get the pieces working as best I could and if there was still time, go back and iterate to make improvements. There is a saying I am always telling my kids... "Do the best with what you got"... so it was time to take my own advice.

I started my app by making a shell of just the app name and a bin directory--granted Splunk does provide an add-on builder app to help with this but maybe I am just lazy or stubborn as I am yet to use it--and downloaded and placed the nltk directory there. Consequently pypi.org is a great place to manually download Python packages and where I did so (https://pypi.org/project/nltk/). I then changed my directory into the bin folder (I split my development time between a Mac and a Linux server) and ran Splunk's CLI command to get into its Python IDE. The command is `$SPLUNK_HOME/bin/splunk cmd python` (replacing `$SPLUNK_HOME` with the actually directory if this is not in your environment path), and is great way to test out to see if Splunk will succeed in simply importing the library or has further dependancies. 

The first problem I ran into is that it could not import six (https://pypi.org/project/six/), but I have needed that package for other apps before, so I copied it over and tried again. I then received an error of wanting to import sqlite3. This was a bit more confusing as allegedly this is supposed to be packaged in Python since 2.5 which was captured nicely in a stackoverflow (2013) answer https://stackoverflow.com/questions/19530974/how-can-i-install-sqlite3-to-python, Splunk’s mini python environment is currently 2.7.14. However the very next answer from the same post was truly what I needed, which showed me I could download pysqlite to satisfy the condition. However once downloading the files, it wasn’t evident which were the right files, I finally figured out to just grab the folder called “lib” into my app’s bin directory and rename it to sqllite3. Now importing nltk works. 

The next problem for packaging nltk however is that nltk expects certain documents to be downloaded and available. This includes corpora, tagsets, tokenizers and more. In my case though I would want to control what is downloaded and where it is downloaded to. A great Medium article (satoru, 2016) pointed me to the place where all of this data could be downloaded which is http://www.nltk.org/nltk_data/. Though ultimately a stackoverflow.com (2018) answer showed that I could either append the value nltk.data.path or set an environment variable. Once I appended the value and downloaded the data I was able to import stop words successfully. So my custom command would need to add to that path every time.

#### Cleaning the Data

First order of business was creating a new custom command to package with the app for cleaning text. Now I knew that Splunk natively could certainly handle pieces of this already--like lowercase, split words by spaces, remove digits and punctuation--but having a single command to complete this is compelling and useful but more importantly Splunk does not have a native way to complete word stemming or lemmatization (or later I also added the ability to extract ngrams with the same command). Stemming and lemmatization are all about changing words back to their base. Personally I don't really like stemming as often the result is not a real word at all as often it just chops off suffixes but if I am already providing lemmatization maybe someone else has a good reason for it--plus stemming is very fast comparatively. So my command would provide the cleaning options I mentioned earlier, changing words back to their base, and also an option to remove URLs because when you remove the punctuation out of a URL you are left with gibberish. 

The first part of creating a custom command with Splunk's SDK that I would deal with was options. Referring especially to what options the command would take.  I learned that the SDK’s options are quite involved, it has validators that can validate if the option the user gave is any of these:

In [ ]:
        Boolean
        Code
        Duration
        File
        Integer
        List
        Map
        RegularExpression
        Set

I also learned I could set a default for the option with simply providing `default=<myvalue>` and even require the option with `require=True`. Putting it all together, an example option looks like:

In [ ]:
    default_clean = Option(
        default=True,
        doc='''**Syntax:** **lowercase=***<boolean>*
        **Description:** Change text to lowercase, remove punctuation, and removed numbers, defaults to true''',
        validate=validators.Boolean()
        )

The validators for boolean are great in Splunk, because it will take lots of different options to equal true or false like t or true or TRUE or 1 or True. 

I started with just making my command simple and changing text to lowercase by default, I successfully changed it to lowercase based on the options given at the Splunk search input. 

I made a decision that it would be more efficient to lowercase, remove punctuation and numbers all at the same time and that these could be treated as one option (on or off) given that this is something you can already do in Splunk so it is more of a convenience factor more than anything though here I struggled with what to call the combined option: 

std_clean <br/>
basic_wash <br/>
basic_clean <br/>
std_options <br/>
basic_option_group <br/>
default_clean

I settled on the last… default_clean (obviously since that is the example I gave). Actually writing out the possible options was a good exercise, this goes back to the old adage by Phil Karlton “There are only two hard things in Computer Science: cache invalidation and naming things.” (Hilton, 2016), which is so true but also can be pretty important and sometimes difficult to change. I ran into similar difficulty for the other options I needed to name as well and the command ended up with A LOT of options which is especially painful when it comes to documenting the command--something I find that often takes longer to write than the code itself.

I would say throughout the creation of the cleantext command I played tug-of-war with the following:
* order of operations in general
* DRY (Do Not Repeat Yourself) programming versus providing user options
* speed of the command (`set` by the way is awesome, turn a list into a set and Python does not have to look at each element)

Anyway these can have competing interests. During the creation of the command I realized very quick that I would need some quick text to test on (other than using `makeresults` and an `eval`). I had planned on packaging the app with some texts from the Gutenberg library anyway, so I paused to clean and reformat several texts--Peter Pan, Pride and Prejudice, and Moby Dick--the jupyter notebooks are in this same project folder. I made the texts into CSV files, broken up by sentence but also including a title, author, and chapter columns. This way I could easily use Splunk's `inputlookup` command to import them and I used the example text to test out speed. Splunk has an “Inspect Job” feature that tells the overall time but the times can vary based on the business of the computer so I do a couple of takes per trial.

Some other issues I ran into during the creation of this command is parts-of-speech (POS) tagging and its relation to lemmatization. POS tagging is labeling words in their grammatical structure for words such as noun, verb, and so on. NLTK's lemmatization command (`WordNetLemmatizer`) assumes that all words are nouns if you do not feed it the POS tag. The odd part is that the POS tagger nltk provides does not output the tag in a form that the lemmatizer accepts so you have to convert it. Splunk is good about spitting out Python errors to the screen when a failure happens but this isn't always clear what exactly is the cause of the issue... enter logging. Splunk fortunately provides developers lots of references and tools to make them successful and you would expect a log aggregation tool be able to log itself. Sure enough I used their implementation (http://dev.splunk.com/view/logging/SP-CAAAFCN) to help me get around problem spots. Finally I had some issues with tokenizing the words. Here I found that nltk’s word tokenizer does a nice job of breaking the words into a list (which shows up as a multi-valued field in Splunk which was the ideal output). Just splitting on spaces but in the example text would sometimes result in two words on a single space because they were separated by punctuation. To get POS tags I needed to use the word tokenizer but it is certainly slower than splitting by space. I found a new method of Python's regex command--`re.split()`--which does really well with `\W+` as the splitter but then it ends up removing punctuation, so I used that when the user chose not to do lemmatization based on POS.  

At a later stage I really needed a way to handle text that had html markup in it and by handle I mean remove. I had previously used a great Python package called BeautifulSoup (Richardson, 2015) and decided to see if I could in a reasonable amount of time make another custom command. Happily I was able to get it done in time and called the command `bs4` (named after the actual package). It is such a great command that it could really stand as a Splunk app on its own and made short work of removing any html formatting from my support tickets.

#### Welcome Screen

The opening screen of the app is meant to give some quick hits on how to use the custom commands and algorithms that come packaged with app, though I also added links for the extra Splunk apps that it requires to work as well as some basic text analytics terminology. I figured sometimes the language used ("no pun intended") in text analytics can be confusing and it helps to get people on the same page, though I only started with some basics, I may eventually add more.
![welcome_screen](img/welcome_screen.png)


#### EDA - Counts Dashboard

This dashboard of the app would fulfill the exploratory data analysis portion of the class rubric and useful for answering some basic questions about a corpus. Here I made good use of Splunk's ability to use base searches behind the scenes to power multiple panels of the dashboard. Honestly I am of the belief that dashboards are the most powerful aspect of Splunk and possibly the least used, this is because you essentially can create very complex "code" (okay it is really SPL and XML underneath but really SPL is a coding language of its own accord) and then that code can be called again easily and quickly. In fact even for those very experienced in SPL too can benefit from the dashboard because you can run multiple searches at once and--back to the complex aspect--not have to remember how you did something before. Anyway a base search can pull back a large dataset and the panels can make small adjustments to portions of said dataset to visualize it. 

The concept of the counts dashboard is just as the name implies, tokenize the words in the corpus and get counts of how frequently they show up in the text. Of course counts also includes several other counts--total counts of all words, unique words, total counts of documents, ngrams totals (based on the range set, if set that is), counts by parts-of-speech tags, a word cloud of the counts and ngrams if chosen, as well as counts by an optional grouping field. This can result in some interesting data. There also are some drill down options and zipf plot if the user is including stopwords. While working on this dashboard I ended up going back and adding a new output on my cleantext command to get the POS tag and word together as a tuple as well as add an option to provide custom stopwords. Every corpus is going to have some level of common words that are not as useful to that domain, so it makes sense to include this. The Zipf distribution can actually help point these common words out but TFIDF also has an option to remove what they call "corpus-specific stop words" (scikit-learn developers, 2017) using the `max_df` option but I am not using TFIDF during this exploratory phase.

Running the dashboard on one of the sample texts I included with the app--Moby Dick--we see unsurprisingly that "whale" is the top word. Though that doesn't provide much insight, there are some interesting stats here. The unique words found (even with the words lemmatized) is 12,591. This is a very large vocabulary, and this can be seen further in the POS tagging.
![moby_dick_counts](img/moby_dick_counts.png)

Clicking on the POS tag NN, we get an explanation of singular nouns from Dipanjan Sarkar's book _Text Analytics with Python: A Practical Real-World Approach to Gaining Actionable Insights from your Data._ (2016) and see that 686 of the 1299 whale mentions where used as a singular noun. As mentioned before the book has a large vocabulary and this can be seen in Herman Melville's writing style as there are a lot of adjectives (JJ) and adverbs (RB), verbs (VBD) don't even show up until the sixth ranking position.
![moby_dick_NN](img/moby_dick_NN.png)

Clicking on "ahab" we can see where in the text the occurrences of this show up. Sometimes this can be useful when you aren't sure about a term, such as how it is being used.
![moby_dick_search](img/moby_dick_search.png)

Looking at six months of support tickets and removing some custom stopwords and html formatting (this is because many of these tickets come through as emails so the signatures cause a lot of unnecessary repeating words and many emails are html formatted). Nice to see that people are still very polite with "please" being the top word. 
![support_tickets_counts_NN](img/support_tickets_counts_NN.png)

However "please" gets incorrectly interpreted often as a noun. One factor that might cause this is that when users create support tickets they do not necessarily follow grammatical correctness, it might be akin to a text message (i.e. "printer broke, please fix"). Though also given that noun is the default POS tag, anything the tagger cannot interpret ends up a noun (which are the top two columns). So is this perfect? No not by far, but the point here is gain insight and sometimes things like this say "nothing to see here... move along". Though this would be a place that I do miss spaCy which is really good with lemmatization and POS tagging.

Since I chose a grouping field I am presented with a bar graph of quantity of terms per field. In this case I chose "Categories". Looking at the "Other" category, we see a possible dumping ground of the unknown. This is probably used because we don't want our ticket creating options to be too deep. This however is a great opportunity for unsupervised machine learning which could possibly find clusters of topics within something like this.
![support_tickets_group](img/support_tickets_group.png)

At the bottom of the Counts dashboard, we get presented with a couple of word clouds. A couple of things that stand out to me is users are needing help with email, passwords, and directory access. A word cloud can be more useful than just a table with terms as the eye may catch things it otherwise would not have. 
![support_tickets_wordcloud](img/support_tickets_wordcloud.png)

Looking at the support tickets with stopwords left in tact, interestingly we see that "to" is the top word. Normally "the" is the most common word, but this might again go back to the language norms surrounding opening support tickets. Though Splunk does not currently have a visualization that supports both logarithmic scales and points with a line through it, so I could not point the distribution line with it. However without the stopwords the tickets do appear to follow the Zipf distribution as expected.

![support_tickets_zipf](img/support_tickets_zipf.png)

#### Unsupervised Learning - Themes Dashboard

Building out the Themes dashboard, the concept was to walk a user through the various options in order for the dashboard to do unsupervised learning on the text and display the clusters in various ways. I went ahead and added every clustering algorithm that MLTK offers, however not all of these are necessarily ideal for text. Though when you come down to it, text analytics and unsupervised learning in general, is not an exact science

#### Sentiment Analysis - Sentiment Dashboard

No text analytics would be complete without some sort of sentiment analysis. The odd part however is sentiment is an oddity for how it fits into data science and the data science project lifecycle. One the one hand it can be and end in itself and something that someone is specifically after or wants to know. Its results might cause a new question, such as why was the sentiment slow low or high warranting new investigations. On the other hand it can be used to add features to a data set which could be used as a predictor or it can simply be exploratory in nature in that you are just interested in what analysis sentiment might render. Some sentiment analysis engines are even machine learning models under the hood. However knowing I wanted to add sentiment to this Splunk app, I knew I wanted to use VADER which is a rule-based sentiment engine (Hutto & Gilbert, 2014). VADER--which what cool name for a command, you can almost queue the music--stands for Valence Aware Dictionary for sEntiment Reasoning.

#### Conclusion

I set out to bring natural language processing to Splunk... did I accomplish this? Yeah I think so. Though lots of bumps and bruises along the way and definitely still room for improvement. Text analytics is hard. This probably comes down to the fact that once the data is tokenized, there are just so many features per document to begin with. Not to mention language has so many nuisances.

### References
See https://github.com/geekusa/nlp-text-analytics/blob/master/PROJECT_FILES/References.md for the list of references for this paper and project.